### Лекция и семинар 11. Юнит-тестирование

<br />

##### Основы

Тестирование по целям:
* functional
* performance
* usability
* ...

functional-тесты:
* юнит-тесты
* интеграционные тесты

__Вопросы-набросы__:

<details>
<summary>Зачем нужны юнит-тесты?</summary>
<p>

* Порверка корректности на этапе написания
* Предотвращение поломки после рефакторинга

</p>
</details>

<details>
<summary>Что тестировать?</summary>
<p>

* success path
* failure path
* граничные значения
* различные пути выполнения (см. [цикломатическая сложность](https://en.wikipedia.org/wiki/Cyclomatic_complexity))

</p>
</details>

Чем тестировать:

Наиболее популярной библиотекой для юнит-тестирования является [google test](https://github.com/google/googletest) и набор вспомогательных утилит для неё [google mock](https://github.com/google/googletest/tree/master/googlemock).

Правила именования:

```c++
TEST(TestSuiteName, TestName)
```

Примеры:

```c++
// sqrt_unittest.cpp

double sqrt(double x);

TEST(sqrt, zero)
{
    double v = sqrt(0.);
    EXPECT_NEAR(v, 0., 1e-6);
}

TEST(sqrt, nine)
{
    EXPECT_NEAR(sqrt(9.), 3., 1e-6);
}
```

```c++
// fabs_unittest.cpp

double fabs(double x);

TEST(fabs, zero)
{
    EXPECT_EQ(fabs(0.), 0.);
}

TEST(fabs, positive)
{
    EXPECT_EQ(fabs(3.), 3.);
}

TEST(fabs, negative)
{
    EXPECT_EQ(fabs(-3.), 3.);
}
```

Библиотека gtest генерирует имена тестов по правилу:

полное имя теста = `TestSuiteName.TestName`

В примере будут сгенерированы тесты:

* `sqrt.zero`
* `sqrt.nine`
* `fabs.zero`
* `fabs.positive`
* `fabs.negative`

Макрос `TEST` помимо генерации тела теста ещё регистрирует тело теста и его имя в глобальном контейнере тестов. Затем запускалка перебирает глобальный контейнер и запускает тесты один за другим.

В файле `sqrt_unittest.o` после компиляции будет сгенерирован код регистрации тестов в глобальном контейнере:
* `sqrt.zero`
* `sqrt.nine`

В файле `fabs_unittest.o` после компиляции будет сгенерирован код регистрации тестов в глобальном контейнере:
* `fabs.zero`
* `fabs.positive`
* `fabs.negative`

Далее линковщик скомпонует `sqrt_unittest.o` и `fabs_unittest.o` вместе в один исполняемый файл, в котором будет код регистрации для каждого теста.

__Замечание__: Чтобы добавить тесты на функцию `pow` нужно...

<details>
<summary>...?</summary>
<p>

создать аналогичный файлик `pow_unittest.cpp` и добавить его в компиляцию и линковку.

</p>
</details>


<br />

Запуск:

`./project_unittest`

Можно пофильтровать по именам:

`./project_unittest --gtest_filter=sqrt.*`

__Вопрос__:

<details>
<summary>как прогнать тесты sqrt.zero и fabs.zero?</summary>
<p>
    
`./project_unittest --gtest_filter=*.zero`
    
</p>
</details>

<br />

##### Идеалистическая организация теста

```c++
TEST(function, scenario)
{
    // настройка окружения
    ...

    // выполнение действия, результат которого надо протестировать
    ...
    
    // проверка условий (в идеале - одного-единственного условия)
    ...
}
```

Пример:

```c++
TEST(read_n, reads_correct_value)
{
    // prepare
    std::stringstream ss("42");
    
    // run action
    const unsigned n = read_n(ss);
    
    // check assertion
    EXPECT_EQ(n, 42);
}
```

Для функционала сложнее калькулятора бывает трудно организовать тесты идеальным образом. И это естественно.

<br />

##### EXPECT_* / ASSERT_*

__Вопрос__:

<details>
<summary>Чем EXPECT_EQ отличается от ASSERT_EQ?</summary>
<p>

В случае несовпадения значений
* `EXPECT_EQ` генерирует нефатальную ошибку и продолжает выполнение теста
* `ASSERT_EQ` генерирует фатальную ошибку - останавливает выполнение теста

</p> 
</details>

Для чистоты кода и более адекватных сообщениях об ошибке существуют разные варианты `EXPECT_*` и `ASSERT_*`:
    
* булевы выражения
    ```c++
    EXPECT_TRUE(v.empty());
    EXPECT_FALSE(v.empty());
    ```
* равенство (для вещ. чисел см. [статью](https://randomascii.wordpress.com/2012/02/25/comparing-floating-point-numbers-2012-edition/))
    ```c++
    EXPECT_EQ(x, y);
    EXPECT_NE(x, y);
    EXPECT_FLOAT_EQ(x, y);
    EXPECT_DOUBLE_EQ(x, y);
    EXPECT_NEAR(x, y, abs_error);  // <-- use this for float/double values
    ```
* порядок
    ```c++
    EXPECT_LT(x, y);
    EXPECT_LE(x, y);
    EXPECT_GT(x, y);
    EXPECT_GE(x, y);
    ```
* C-строки
    ```c++
    const char* s = "Get the rack!";
    EXPECT_STREQ(s, "Nobody expects for Spanish Inquisition!");
    EXPECT_STRNE(s, "Nobody expects for Spanish Inquisition!");
    EXPECT_CASEEQ(s, "Nobody expects for Spanish Inquisition!");
    EXPECT_CASENE(s, "Nobody expects for Spanish Inquisition!");
    ```
* исключения
    ```c++
    EXPECT_THROW(my_function(3., 4.), std::runtime_error);
    EXPECT_ANY_THROW(my_function(3., 4.));
    EXPECT_NO_THROW(my_function(3., 4.));
    EXPECT_NO_THROW({
        const double x = get_number();
        my_function(x);
    });
    ```
* кастомные условия (можно через `EXPECT_TRUE`, но эти варианты выведут аргументы в значения об ошибке)
    ```c++
    bool isPrime(int n) { ... }
    bool isMutuallyPrime(int m, int n) { ... }

    EXPECT_PRED1(isPrime, 5);
    EXPECT_PRED2(isMutuallyPrime, 9, 10);
    ```
* матчеры из gmock ... (о них в продвинутой части)

<br />

##### ASSERT_NO_FATAL_FAILURE

`ASSERT_*` - проверки на фатальную ошибку. Делает 3 вещи:
* печатают сообщение об ошибке
* добваляют информацию о фатальной ошибке (например, в глобальную (или `thead_local`) переменную)
* `return` из текущей функции (функция должна быть `void`)

`ASSERT_NO_FATAL_FAILURE(statement)` - проверка, что `statement` не добавил новых фатальных ошибок, если добавил - `return` из текущей функции (д.б. `void`)

Зачем?

```c++
void setup_testing_environment()
{
    const bool network_ok = setup_testing_network();
    ASSERT_TRUE(network_ok);
    
    const bool database_ok = setup_testing_database();
    ASSERT_TRUE(database_ok);
}

TEST(suite, scenario)
{
    ASSERT_NO_FATAL_FAILURE(setup_testing_environment());
    
    ...
}
```

<br />

##### Fixtures

Пример:

Пишем тесты для функции поиска кратчайшего пути в графе

`Way dijkstra(const Graph& g, const Vertex& v_start, const Vertex& v_final);`

Для серии тестов на `dijkstra` нужно создавать один и тот же граф `G` через вызовы:

```c++
auto V = make_special_vertices();
auto E = make_special_edges(V);
auto G = make_graph(V, E);
```

Как могли бы выглядеть тесты:
    
```c++
TEST(dijkstra, same_vertex)
{
    // setup special graph
    const auto V = make_special_vertices();
    const auto E = make_special_edges(V);
    const auto G = make_graph(V, E);
    
    // way v1 -> v1 should have zero len
    const auto v1 = find_vertex(V, "v1");
    const auto way = dijkstra(G, v1, v1);
    EXPECT_EQ(way.length(), 0);
}

TEST(dijkstra, unreachable_vertex)
{
    // setup special graph
    const auto V = make_special_vertices();
    const auto E = make_special_edges(V);
    const auto G = make_graph(V, E);
    
    // way between unreachable vertices shouldn't exist
    const auto v1 = find_vertex(V, "v1");
    const auto v9 = find_vertex(V, "v9");    
    const auto way = dijkstra(G, v1, v9);
    EXPECT_FALSE(way.is_finite());
}

TEST(dijkstra, normal_way)
{
    // setup special graph
    const auto V = make_special_vertices();
    const auto E = make_special_edges(V);
    const auto G = make_graph(V, E);
    
    // normal way should be ok
    const auto v1 = find_vertex(V, "v1");
    const auto v2 = find_vertex(V, "v2");
    const auto way = dijkstra(G, v1, v2);
    EXPECT_TRUE(way.is_finite());
}
```

__Вопрос__:

<details>
<summary>В чём проблема? Что должно моментально вызывать аллергическую реакцию программиста?</summary>
<p>

copy-paste

</p>
</details>

Вариант починить проблему - fixtures.

Fixtures позволяют единообразно и разово задать:
* необходимые действия для инициализации теста
* необходимые действия для очистки теста

Пример:

```c++
class DijstraSpecialGraph : public Test
{
protected:
    void SetUp() override
    {
        V_ = make_special_vertices();
        E_ = make_special_edges(V_);
        G_ = make_graph(V_, E_);    
    }
    
    const Vertices& vertices() const noexcept { return V_; }
    const Edges& edges() const noexcept { return E_; }
    const Graph& graph() const noexcept { return G_; }
    
private:
    Vertices V_;
    Edges E_;
    Graph G_;
};

TEST_F(DijstraSpecialGraph, same_vertex)
{
    const auto v1 = find_vertex(vertices(), "v1");
    const auto way = dijkstra(graph(), v1, v1);
    EXPECT_EQ(way.length(), 0);
}

TEST_F(DijstraSpecialGraph, unreachable_vertex)
{
    const auto v1 = find_vertex(vertices(), "v1");
    const auto v9 = find_vertex(vertices(), "v9");
    const auto way = dijkstra(graph(), v1, v9);
    EXPECT_FALSE(way.is_finite());
}

TEST_F(DijstraSpecialGraph, normal_way)
{
    const auto v1 = find_vertex(vertices(), "v1");
    const auto v2 = find_vertex(vertices(), "v2");
    const auto way = dijkstra(graph(), v1, v2);
    EXPECT_TRUE(way.is_finite());
}
```

Обратите внимание на:
* fixture определяется как класс
* используется макрос `TEST_F`
* В fixture можно определить методы `void SetUp()` и `void TearDown()`
    * `SetUp` - действия до запуска теста
    * `TearDown` - действия после прогона теста

__Вопрос__:

<details>
<summary>Что делать, если нужно 10 тестов на специальный граф и 10 тестов на специальный граф чуть-чуть другого вида, например, с добавлением одного ребра для создания цикла?</summary>
<p>
    
Варианты:
* иерархия классов `DijstraSpecialGraph`: `DijstraSpecialGraphBase` + `DijstraSpecialGraphAcyclic` + `DijstraSpecialGraphCyclic`:

    ```c++
    class DijstraSpecialGraphBase : public ::testing::Test
    {
    protected:
        DijstraSpecialGraphBase(bool isCycled) { ... }
        ...
    };

    class DijstraSpecialGraphAcyclic : public DijstraSpecialGraphBase
    {
    protected:
        DijstraSpecialGraphAcyclic() : DijstraSpecialGraphBase(false) {}
    };

    class DijstraSpecialGraphCyclic : public DijstraSpecialGraphBase
    {
    protected:
        DijstraSpecialGraphCyclic() : DijstraSpecialGraphBase(true) {}
    };
    ```
    
* шаблонный fixture (почему бы и нет?):

    ```c++
    template<bool IsCyclic>
    class DijstraSpecialGraph : public ::testing::Test
    {
        ...
    };

    using DijstraSpecialGraphAcyclic = DijstraSpecialGraph<false>;
    using DijstraSpecialGraphCyclic = DijstraSpecialGraph<true>;
    ```

* value-parametrized tests (позже и посложнее)

</p>
</details>

__Замечания__:
* Из-за организации gtest внутри тела теста можно доступаться только до protected данных fixture, т.к. gtest генерирует неявного наследника.
* В случае иерархии fixture protected поля - сомнительная идея, возможно, лучше защитить их через protected - аксессоры, ещё лучше - const-аксессоры

<br />

Как это работает для каждого теста:
* Сохранить состояния флагов googletest
* Создать объект fixture
* Позвать метод `SetUp` у fixture
* Прогнать тест
* Позвать `TearDown` у fixture
* Удалить объект fixture.
* Восстановить состояния флагов.

```c++
    save_gtest_flags();
    {
        Fixture f;
        f.SetUp();
        f.RunTest();
        f.TearDown();
    }    
    restore_gtest_flags();
```

__Вопрос__: Сколько объектов fixture здесь будет создано?
    
```c++
TEST_F(DijstraSpecialGraph, same_vertex)
{
}

TEST_F(DijstraSpecialGraph, unreachable_vertex)
{
}

TEST_F(DijstraSpecialGraph, normal_way)
{
}
```

<br />

##### Отключение тестов

Если тест падает, но чинить прямо сейчас его некогда (в больших проектах бывает), его можно отключить:

```c++
TEST(DijstraSpecialGraph, DISABLED_unreachable_vertex) { ... }
```

Или отключить всю fixture:

```c++
class DISABLED_DijstraSpecialGraph : public Test { ... };

TEST_F(DISABLED_DijstraSpecialGraph, same_vertex)
{
}

TEST_F(DISABLED_DijstraSpecialGraph, unreachable_vertex)
{
}

TEST_F(DISABLED_DijstraSpecialGraph, normal_way)
{
}
```

__Вопрос__: Вариант решения проблемы - закомментировать тест, но лучше применять DISABLED, почему?

Как отключить тест для платформы?

```c++
#ifdef __WINDOWS__
#define MAYBE_normal_way DISABLED_normal_way
#else
#define MAYBE_normal_way normal_way
#endif

TEST_F(DijstraSpecialGraph, MAYBE_normal_way)
{
}
```

<br />

##### Запуск тестов, доп. опции

Фильтрация тестов:

* запустить только сюиту `sqrt`:

    `./unittests --gtest_filter=sqrt.*`

* запустить `sqrt` && `fabs`:

    `./unittests --gtest_filter=*sqrt*:*fabs*`

* запустить `sqrt`, но не `sqrt.zero`:

    `./unittests --gtest_filter=sqrt.*-sqrt.zero`

* `sqrt` и `fabs`, но не `zero`:

    `./unittests --gtest_filter=sqrt.*:fabs.*-sqrt.zero:fabs.zero`

Многократный запуск тестов:

`./unittests --gtest_repeat=1000`

__Вопрос__: зачем?

Остановиться на первом падении:

`./unittests --gtest_break_on_failure`

__Вопрос__: зачем?

Перемешать порядок тестов:

`./unittest --gtest_shuffle --gtest_random_seed=100500`

__Вопрос__: зачем?

Изменение формата вывода:
    
`./unittest --gtest_output=json|xml`

__Вопрос__: зачем?

<br />

##### Что делать, если нужно протестировать состояние приватного поля класса?

```c++
class TriangleShape
{
public:
    TriangleShape();
    void draw();

private:
    Color color_;      // <-- need to test this value
    Point v1, v2, v3;  // <-- need to test this value
};
```

<details>
<summary>Ответ</summary>
<p>

страдать

</p>
</details>

Вариант 0: выводить в public всё нужное (совсем плохо)

Вариант 1: `_for_testing` - методы

```c++
class TriangleShape
{
public: // usual interface
    TriangleShape();

    void draw();
    
public: // testing-only methods
    const Color& get_color_for_testing() const { return color_; }
    const Point& get_vertex_1_for_testing() const { return v1; }
    const Point& get_vertex_2_for_testing() const { return v2; }
    const Point& get_vertex_3_for_testing() const { return v3; }

private:
    Color color_;
    Point v1, v2, v3;
};
// проще отследить использование "неправильных" методов в боевом коде, но:
//   а) много лишнего шума в классе (читабельность, вреся компиляции)
//   б) лишние методы компилируются в исполняемый файл
```

Вариант 2: `FRIEND_TEST`

```c++
class TriangleShape
{
public: // usual interface
    TriangleShape();

    void draw();

private:
    // перечислим тесты, которым можно залезать в private:
    FRIEND_TEST(TriangleShape, TestName1);
    FRIEND_TEST(TriangleShape, TestName2);
    FRIEND_TEST(TriangleShape, TestName3);
    FRIEND_TEST(AnotherSuite, AnotherTestName1);
    FRIEND_TEST(AnotherSuite, AnotherTestName2);
    
    Color color_;
    Point v1, v2, v3;
};
// в боевом коде лишние методы не компилируется, но:
//   а) много лишнего шума в классе (больше тестов - длиннее текст класса)
//   б) в итоговой программе нужен инклуд из тестовой библиотеки
```

Вариант 3: friend-обёртка над классом для тестов

```c++

// production-код:

class TriangleShape
{
public: // usual interface
    TriangleShape();

    void draw();

private:
    friend class TriangleShapeTestingAccessor;

    Color color_;
    Point v1, v2, v3;
};

// код, компилирующийся только в рамках юнит-тестов:

class TriangleShapeTestingAccessor
{
public:
    static const Color& get_color(const TriangleShape& shape) { return shape.color_; }    
};

// сами тесты:

TEST(Suite, Name)
{
    TriangleShape tri = make_triangle();
    const auto color = TriangleShapeTestingAccessor::get_color(tri);
}

// в боевом коде почти нет артефактов от тестирования, но:
//     а) приходится писать отдельный TestingAccessor на каждый сложно тестируемый класс
```

<br />

##### Матчеры (gmock)

Матчеры - расширение набора проверок `EXPECT_*`

https://github.com/google/googletest/blob/master/googlemock/docs/cheat_sheet.md#matchers-matcherlist

Вызов:
    
```c++
EXPECT_THAT(actual_value, matcher)
ASSERT_THAT(actual_value, matcher)
```

Пример:

```c++
TEST(...)
{
    std::vector<int> numbers = make_numbers();
    EXPECT_THAT(numbers, IsEmpty());
}
```

Какие матчеры бывают:

* аналоги обычных `EXPECT_*/ASSERT_*`
    * `EXPECT_THAT(value, Eq(42));`
    * `EXPECT_THAT(value, Ne(42));`
    * `EXPECT_THAT(value, Ge(42));`
    * `EXPECT_THAT(value, Gt(42));`
    * `EXPECT_THAT(value, Le(42));`
    * `EXPECT_THAT(value, Lt(42));`
    * `EXPECT_THAT(value, IsTrue());`
    * `EXPECT_THAT(value, IsFalse());`
    * `EXPECT_THAT(value, DoubleEq(42.0));`
    * `EXPECT_THAT(value, FloatEq(42.0));`
    * `EXPECT_THAT(value, DoubleNear(42.0, 1e-3));`
    * `EXPECT_THAT(value, FloatNear(42.0, 1e-3));`
* указатели
    * `EXPECT_THAT(pointer, IsNull());`
    * `EXPECT_THAT(pointer, NotNull());`
* строки
    * `EXPECT_THAT(s, EndsWith("abc"));`
    * `EXPECT_THAT(s, StartsWith("abc"));`
    * `EXPECT_THAT(s, HasSubstr("abc"));`
    * `EXPECT_THAT(s, StrEq("abc"));`
    * `EXPECT_THAT(s, StrNe("abc"));`
    * `EXPECT_THAT(s, StrCaseEq("abc"));`
    * `EXPECT_THAT(s, StrCaseNe("abc"));`
    * `EXPECT_THAT(s, ContainsRegex("abc*"));`
* контейнеры
    * `EXPECT_THAT(numbers, IsEmpty());`
    * `EXPECT_THAT(numbers, SizeIs(5));`
    * `EXPECT_THAT(numbers, ContainerEq(another_numbers));`
    * `EXPECT_THAT(numbers, Contains(42));`
    * `EXPECT_THAT(strings, Each(IsEmpty()));`
    * `EXPECT_THAT(numbers, ElementsAre(1, 2, 3, 4, 5));`
    * `EXPECT_THAT(numbers, ElementsAreArray(another_numbers));`
    * `EXPECT_THAT(numbers, UnorderedElementsAre(1, 2, 3, 4, 5));`
    * `EXPECT_THAT(numbers, UnorderedElementsAreArray(another_numbers));`
    * `EXPECT_THAT(numbers, IsSubsetOf({1, 2, 3, 4, 5}));`
    * `EXPECT_THAT(numbers, WhenSorted(ElementsAre(1, 2, 3)));`
* композиции
    * `AllOf`:
    ```c++
    EXPECT_THAT(string,
                AllOf(EndsWith(".txt"),
                      SizeIs(10)));
    ```
    * `AnyOf`:
    ```c++
    EXPECT_THAT(string,
                AnyOf(EndsWith(".txt"),
                      SizeIs(10)));

    ```
    * `Not`:
    ```c++
    EXPECT_THAT(numbers, Not(IsSubsetOf({1, 2, 3, 4, 5})));
    ```
* создание собственных матчеров:

    ```c++
    MATCHER(IsEven, "") { return (arg % 2) == 0; }

    EXPECT_THAT(value, IsEven());
    ```

__Упражение__:

<details>
<summary>С помощью матчеров проверить, что в списке имён файлов нет текстовых (.txt)</summary>
<p>

`EXPECT_THAT(filenames, Each(Not(EnsWith(".txt"))));`
    
</p>    
</details>

<br />

##### Dependency inversion

Инверсия зависимостей - техника, позволяющая "развязать" зависимости между компонентами программы через интерфейсы.

https://en.wikipedia.org/wiki/Dependency_injection

https://en.wikipedia.org/wiki/Dependency_inversion_principle

Рассмотрим на примере 2 реализации поиска самого молодого студента в БД:

Вариант 1:

```c++
std::optional<Student> find_youngest_student(const Database& db)
{
    std::optional<Student> rv;
    
    const auto students = db.get_students();
    if (!students.empty())
    {
        rv.emplace(*min_element(begin(students), end(students),
                                [](const Student& lhs, const Student& rhs){
                                    return lhs.age < rhs.age;
                                }));
    }
     
    return rv;
}
```

Вариант 2:
    
```c++

// IDatabase.h
// где-то в районе определения кода БД:
struct IDatabase
{
    virtual ~IDatabase() = default;
    
    virtual std::vector<Student> get_students() const = 0;
};

// Database.h
// где-то в районе определения кода БД:
class Database : public IDatabase
{
public:
    std::vector<Student> get_students() const override;
    ...
};

// функция поиска самого молодого студента
// Обратите внимание на тип принимаемого аргумента

std::optional<Student> find_youngest_student(const IDatabase& db)
{
    std::optional<Student> rv;

    const auto students = db.get_students();
    if (!students.empty())
    {
        auto it = min_element(begin(students), end(students),
                              [](const Student& lhs, const Student& rhs){
                                  return lhs.age < rhs.age;
                              });
        rv.emplace(*it);
    }

    return rv;
}
```

__Особенности__:
* Сложность реализации:

    Вариант с интерфейсом посложнее (нужно писать интерфейсы и наследование)
    
    
* Время выполнения:

    Вариант с интерфейсом содежрит доп. расходы на вызов виртуальной функции
    
    
* Время компиляции:

    В среднем по больнице, вариант с интерфейсом компилируется быстрее
    
<details>
<summary>Почему? что увеличивает, а что уменьшает время компиляции?</summary>
<p>
    
увеличивает время компиляции:
* парсинг самого интерфейса при компиляции кода БД
* парсинг с наследованием и накладные расходы на него
* генерация доп. кода прыжков по вирутальным функциям
    
уменьшает время компиляции (т.к. внешний код к БД инклудит только IDatabase.h):
* не нужно парсить и компилировать "..." в классе `Database`
* из-за этого `IDatabase.h` (как правило) тянет много меньше зависимостей по инклудам
* не нужно перекомпилировать при изменении `Database.h`
    
</p>
</details>
    
* Тестируемость:

    Вариант 1 тестировать нетривиально, вариант 2 - очень легко:

```c++
class EmptyTestingDatabase : public IDatabase
{
public:
    std::vector<Student> get_students() const override
    {
        return {};
    }
};

TEST(find_youngest_student, no_students)
{
    EmptyTestingDatabase db;
    
    const auto maybe_student = find_youngest_student(db);
    
    EXPECT_FALSE(maybe_student.has_value());
}

class SingleEinsteinDatabase : public IDatabase
{
public:
    std::vector<Students> get_students() const override
    {
        return {Student{"Einstein", 38}};
    }        
};

TEST(find_youngest_student, those_boy_einstein)
{
    SingleEinsteinDatabase db;
    
    const auto maybe_student = find_youngest_student(db);
    ASSERT_TRUE(maybe_student.has_value());
    
    EXPECT_EQ(maybe_student.value().name == "Einstein");
}
```

<br />

##### Mocks

Мокирование - механизм, который помогает на лету генерировать вспомогательные тестовые объекты типа `EmptyTestingDatabase`, `SingleEinsteinDatabase`.

Пример:

```c++
class MockDatabase : public IDatabase
{
public:
    MOCK_METHOD(std::vector<Student>, get_students, (), (const, override));        
};


TEST(find_youngest_student, no_students)
{
    MockDatabase db;
    ON_CALL(db, get_students())
        .WillByDefault(Return(std::vector<Student>()));

    const auto maybe_student = find_youngest_student(db);

    EXPECT_FALSE(maybe_student.has_value());
}

TEST(find_youngest_student, those_boy_einstein)
{
    MockDatabase db;
    ON_CALL(db, get_students())
        .WillByDefault(Return(std::vector<Student>{Student{"Einstein", 38}}));

    const auto maybe_student = find_youngest_student(db);
    ASSERT_TRUE(maybe_student.has_value());

    EXPECT_EQ(maybe_student.value().name == "Einstein");
}
```

<br />

2 варианта работы с mock-объектами:
* `ON_CALL` - указать, что будет происходить при вызове метода
* `EXPECT_CALL` - указать ожидания к вызову (expectations) и что   

Примеры с `EXPECT_CALL` от gtest-a:

```c++
EXPECT_CALL(turtle, GetX())
    .Times(5)
    .WillOnce(Return(100))
    .WillOnce(Return(150))
    .WillRepeatedly(Return(200));
// TODO: что произойдёт, если в тесте GetX позвался 4 раза?
    
// что здесь?
EXPECT_CALL(turtle, GoTo(50, _))
    .Times(AtLeast(3));

// что здесь?
EXPECT_CALL(turtle, Forward(Ge(100)))
    .Times(Between(2, 5));

// что ожидается здесь?
EXPECT_CALL(turtle, GetY())
    .Times(4)
    .WillOnce(Return(100));
```

<details>
<summary>Ответ</summary>
100, 0, 0, 0
</details>

Mock-объектам можно задать действия, что делать при срабатывании вызова:

```c++
TEST(find_youngest_student, those_boy)
{
    MockDatabase db;
    ON_CALL(db, get_students())
        .WillOnce(Invoke([]() -> std::vector<Student> {
            std::cout << "get_students() called" << std::endl;
            return {Student{"Einstein", 38}};
        }));
    ...
}
```

Ещё больше способов задать хитрых действий:
https://github.com/google/googletest/blob/master/googlemock/docs/cheat_sheet.md#actions-actionlist

Гайд от gtest-а по работе с mock-объектами:
1. создать mock-объект
2. если требуется, задать дефолтные actions
3. задать expectations
4. выполнить код теста
5. при уничтожении mock-объекта автоматически проверяется, что все его expectations выполнены

<br />

##### Тестирование последовательности вызовов методов mock-объекта

Mock-объектам можно указывать, в каком порядке должны вызываться методы.

Для этого используется объект класса `InSequence`.

Пример от gtest-а:

```c++
TEST(FooTest, MoveTurtle)
{
  MockTurtle turtle;

  InSequence seq;
  EXPECT_CALL(turtle, PenDown());
  EXPECT_CALL(turtle, Forward(100));
  EXPECT_CALL(turtle, PenUp());

  RunScenario(turtle);
}
```

Пример на псевдокоде, где такое требование может быть логичным:

```c++
TEST(LocationService, LocationIsRequested)
{
    MockLocationService location_service;
    
    InSequence seq;
    EXPECT_CALL(location_service, Initialize());
    EXPECT_CALL(location_service, RequestLocation())
        .WillByDefault( 
            Return(
                Location(55, 83)));
    EXPECT_CALL(location_service, Shutdown());
    
    const auto app = LaunchApp();    
    DisplayCurrentLocation(app);
    CheckLocationDisplayed(app);
}
```

Более сложный пример с графом зависимостей:

```c++
Expectation init_x = EXPECT_CALL(obj, InitX());
Expectation init_y = EXPECT_CALL(obj, InitY());
EXPECT_CALL(obj, some_method())
     .After(init_x, init_y);
```

Больше примеров в документации:
    
https://github.com/google/googletest/blob/master/googlemock/docs/cheat_sheet.md#expectation-order

<br />

##### Тесты, параметризированные значениями

Пример, найденный на просторах сети, на мой субъективный взгляд, исключительно демонстрационный и сверх меры усложняющий альтернативное "тупое" решение:

Вариант 1:

```c++
class LeapYearTest : public TestWithParam<int> {};

TEST_P(LeapYearTest, LeapYear) {
    const int year = GetParam();  // <-- notice GetParam() call
    EXPECT_TRUE(isLeapYear(year));
}

INSTANTIATE_TEST_CASE_P(
        LeapYearTestsP,
        LeapYearTest,
        Values(104, 1996, 1960, 2012));


class NotLeapYearTest : public TestWithParam<int> {};

TEST_P(NotLeapYearTest, NotLeapYear) {
    const int year = GetParam();  // <-- notice GetParam() call
    EXPECT_FALSE(isLeapYear(year));
}

INSTANTIATE_TEST_CASE_P(
        NotLeapYearTestsP,
        NotLeapYearTest,
        Values(103, 1995, 1961, 1900));
```

Вариант 2:

```c++
struct LeapYearTestParam
{
    int year;
    bool leap;
};

class LeapYearTest : public TestWithParam<LeapYearTestParam> {};

TEST_P(LeapYearTest, ChecksIfLeapYear) {
    EXPECT_EQ(GetParam().leap, isLeapYear(GetParam().year));
}

INSTANTIATE_TEST_CASE_P(
        LeapYearTests,
        LeapYearTest,
        Values(LeapYearTestParam{7, false},
               LeapYearTestParam{2001, false},
               LeapYearTestParam{1996, true},
               LeapYearTestParam{1700, false},
               LeapYearTestParam{1600, true}));
```

__Замечание__: ещё есть тесты, параметризованные типами (на шаблонах)
https://github.com/google/googletest/blob/master/googletest/samples/sample6_unittest.cc

<br />

##### шаринг данных в fixture, глобальный шаринг данных

__Вопрос:__ сколько будет создано объектов класса `DijstraSpecialGraph` при прогоне таких тестов?

```c++
class DijstraSpecialGraph : public Test
{
    ...
};

TEST_F(DijstraSpecialGraph, same_vertex) { ... }
TEST_F(DijstraSpecialGraph, unreachable_vertex) { ... }
TEST_F(DijstraSpecialGraph, normal_way) { ... }
```

Это принцип дизайна gtest-а, который (наиболее вероятно) сэкономил море человекочасов, избежав ненужных зависимостей между соседними тестами.

В случае, если заполнение данных для fixture слишком дорого чтобы делать это на каждый тест, можно шарить данные между fixture:

```c++
class DijstraSpecialGraphTest : public Test
{
protected:
    static void SetUpTestSuite()
    {
        shared_graph_.emplace(...);  // setup shared resource
    }

    static void TearDownTestSuite()
    {
        shared_graph_.reset();  // cleanup shared resource
    }

    void SetUp() override { ... }
    void TearDown() override { ... }

    static std::optional<T> shared_graph_;
};
```

<br />

Ещё можно делать глобальный setup-teardown:

https://github.com/google/googletest/blob/master/googletest/docs/advanced.md#global-set-up-and-tear-down

Но при более-менее прямой организации тестов не могу представить зачем это может понадобиться.

<br />

##### Ускорение компиляции mock-объектов

https://github.com/google/googletest/blob/master/googlemock/docs/cook_book.md#making-the-compilation-faster

Mock-объекты компилируются не быстро.

По утвереждению авторов gtest-а основное время компиляции уходит на автогенерированные конструкторы и деструкторы из-за того, что в них нужно генерировать код инициализации и проверки expectations.

Решение для переиспользуемых mock-объектов - компилировать конструкторы и деструкторы отдельно в cpp-файле.

Было:

```c++
// MockNetworkService.h

class MockNetworkService : public INetworkService
{
public:
    MOCK_METHOD(void, initialize, (), (override));
    MOCK_METHOD(std::string, request, (std::string_view), (override));
    MOCK_METHOD(void, shutdown, (), (override));
};
```

Стало:

```c++
// MockNetworkService.h

class MockNetworkService : public INetworkService
{
public:
    MockNetworkService();
    ~MockNetworkService();
    
    MOCK_METHOD(void, initialize, (), (override));
    MOCK_METHOD(std::string, request, (std::string_view), (override));
    MOCK_METHOD(void, shutdown, (), (override));
};

// MockNetworkService.cpp

MockNetworkService::MockNetworkService() = default;
MockNetworkService::~MockNetworkService() = default;
```

__Вопрос:__ сколько раз будет скомпилирован конструктор и деструктор во втором варианте? Почему? А в первом?

<br />

Полезный доп.материал для самостоятельного изучения:
* default actions на вызов мокированного метода
* nice/strict mocks
* зачем нужен RetiresOnSaturation
* тесты, параметризированные типами
* Death tests
* gtest listener API